# FIT5196 Task 1 in Assessment 1
#### Student Name: Dinesh Kesaboina
#### Student ID: 30914345

Date: 13/09/2020

Version: 1.0

Environment: Python 3.7.3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* os (for OS related functionality, eg: read and write operations, included in Anaconda Python 3.7.3) 
* re (for regular expression, included in Anaconda Python 3.7.3) 
* langid (for language classification, <b> not </b> included in Anaconda Python 3.7.3)



<b><font size=4, color="blue">Note : </font></b>
* Langid was downloaded from the terminal for this assessement. 
* All analysis and thought process for the development of this project was based on the sample output file. For this reason, the sample file will be used initially for analysis. 
* Finally, the same functions used to process the sample data will be applied on the full data to obtain the XML file output. 

## 1.  Import libraries 

In this section, you can write your python scripts to parse the correspondiing file.
You should 
* write proper notes for all code block in this notebook using the Markdown cells
* provide proper comment in your scripts
* run all cells to make sure scripts are runable. If the scripts cannot be run by the assessors, they will not be assessed and zero mark will be given to the task.

In [24]:
# Import librariesarequired the text data analysis
import os 
import langid
from langid import classify
import re

## 2. Read files from the directory
In this section we read all the text files from a given directory. The function ```read_files``` takes an OS file path as the argument and returns a list of text files contained in that file path. Here we use ```os``` package's ```listdir``` method. This method returns the names of all files in the directory contained in a list. Next, we use ```os.path``` module's ```join``` method to join the file path and file name to access each text file. We then proceed to check if a given file is a regular file and if it is a text file using the aforementioned module's ```isfile``` and ```endswith``` methods respectively.


In [25]:
def read_files(file_path):
    """Function returns a list of texts containing all the text files in a file path

    Args:
        file_path (str, mandatory) : Path to a file directory.

    Returns:
        list : List of texts in a directory
    """
    list_of_texts = []
    for txt_file in os.listdir(file_path): 
        txt_file = os.path.join(file_path, txt_file)
        if os.path.isfile(txt_file) and txt_file.endswith('.txt'): 
            with open(txt_file, "r") as file:
                content = file.read()
                list_of_texts.append(content)
    return list_of_texts

# Read all files from 
file_path_sample = './part1/'
sample_data = read_files(file_path_sample)

In [26]:
print(f"There are {len(sample_data)} text files in the list. \
There are of the type {type(sample_data[0])}")

There are 13 text files in the list. There are of the type <class 'str'>


In [27]:
sample_data[0]

'{"data":[{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367403953500164","text":"@thehill Trump’s Ridiculous Behavior about Covid19 at Pandemic Briefings Battles https://t.co/4Gjy5tQ4XI https://t.co/8vcxU75Nz4"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367404029153280","text":"Coronavirus : le plan pour la culture d\'Emmanuel Macron https://t.co/0vUpUoXC1O"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367405157466112","text":"‘Complacent’ UK draws global criticism for Covid-19 response https://t.co/Qmd2GSQPY5"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367406243745792","text":"Quatro hospitais particulares de Fortaleza estão com capacidade esgotada na urgência e emergência | Coronavírus - Últimas Notícias sobre o Coronavírus | O POVO Online https://t.co/y9w5t4Dbs9"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367406285762560","text":"@Eglisecatho @Diocese72 +55 (22)981565421\\nDon\'t you have a BRAIN?\\nOnly (martial law)HOME PRISON, during 30 days

## 3. Extract the data tag
We need to extract just the data tag from our text files. In order to extract just that part from the data which is in string format, we write a regex pattern to extract this data. 

<br>
<center><b><font size=5, color="blue"> Regex explanation </font></b></center>
<br>


* ```data_pattern = r'''(?<="data":)\[\{.*\:.*\}\]\,'''```


This regex is designed keeping mind that the data tag starts with a ``` [{ ``` and has some character data followed by a ```:``` (colon) followed by more charcters (```.``` matches any character). We take advantage of the fact that the data tag ends with ```}],``` to design our regex. Notice that we escape the special characters (known as metacharacters) using a backslash. 

Following the creation of this regex, we use ```re``` module's ```search``` method. This method looks through a string and finds the first occurrence of a pattern. As data tag appears only once in each file, this is appropriate. 

In [28]:
def grab_data_tag(list_of_texts):
    """Function returns a list of strings containing all the data tags in the text files

    Args:
        list_of_texts (list, mandatory): List containing all files which are in string format
    
    Returns:
    list: list containing all the data tags for each file
    """
    tweet_data = []
    for text in list_of_texts:
        
        # create date regex to match in text
        data_pattern = r'''(?<="data":)\[\{.*\:.*\}\]\,'''
        matches = re.search(data_pattern, text)
        
        # Remove the last comma at the end
        _data = matches.group()[:-1]
        tweet_data.append(_data)
    
    assert len(list_of_texts) == len(tweet_data), "The number of files does not equal the \
                                                    equal the number of data tags!"
    
    return tweet_data

# test
tweet_data_sample = grab_data_tag(sample_data)

In [29]:
# delete this 
len(tweet_data_sample) == len(sample_data)

True

## 4. Parse text data 
As we can see that from the previous cell, the files in our data are in <b>string</b> format. Let's take a closer look at the first list element.


In [30]:
tweet_data_sample[0]

'[{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367403953500164","text":"@thehill Trump’s Ridiculous Behavior about Covid19 at Pandemic Briefings Battles https://t.co/4Gjy5tQ4XI https://t.co/8vcxU75Nz4"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367404029153280","text":"Coronavirus : le plan pour la culture d\'Emmanuel Macron https://t.co/0vUpUoXC1O"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367405157466112","text":"‘Complacent’ UK draws global criticism for Covid-19 response https://t.co/Qmd2GSQPY5"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367406243745792","text":"Quatro hospitais particulares de Fortaleza estão com capacidade esgotada na urgência e emergência | Coronavírus - Últimas Notícias sobre o Coronavírus | O POVO Online https://t.co/y9w5t4Dbs9"},{"created_at":"2020-05-07T12:05:43.000Z","id":"1258367406285762560","text":"@Eglisecatho @Diocese72 +55 (22)981565421\\nDon\'t you have a BRAIN?\\nOnly (martial law)HOME PRISON, during 30 days, of all

From examining the file, we see that it has two tags : ```data``` and ```errors```. We now need to parse this text to extract textual information from this data. In order to do that, we create four functions : 

* ```get_tweet_ids```: Get all the ids from the text
* ```get_tweet_texts```: Get all the texts from the text
* ```get_tweet_timedates```: Get all the dates and times from the text
* ```get_full_tweets```: Create a full tweet dictionary using the above functions

<br>
<center><b><font size=5, color="blue"> Regex explanation </font></b></center>
<br>


* ```id_pattern = r'(?P<id>(?<="id":\")\d+(?=\"))'```


In order to extract the ids from all the texts we use a named group called ```id``` to capture all the text ids. We use a <b>positive look behind</b> to get the text appears after ```"id":```, we then match one or more digits followed by a <b>positive look ahead</b> that looks for a ```"```(quotation mark) ahead of the match. Note that we escape the question mark using a backslash( \\ ).


* ```(?P<text>(?<="text":\")[^"].*?(?=\"[,}][^\s\d]))```


In order to extract the texts data from all the texts we use a named group called ```text``` to capture all the text data.We use a <b>positive look behind</b> to get the string appearing after ```"text":```, we then match one or more characters that are not quotation marks, we do this using a <b>negated character class</b>. Note the use of question mark( ? )to implement this lazily. We then implement a <b>positive look ahead</b> to look for either a ```"```(quotation mark) or ```}```(closing curly brace) ahead of the match. This is implemented using a character class(square brackets) followed by the  negated character class which matches to anything other than a space and a digit. This last part helps us account for texts in the data which have mutiple quotation marks. 


* ```r'(?P<created_at>(?<="created_at":\")(?:[12]\d{3}-(?:0[1-9]|1[0-2])-(?:0[1-9]|[12]\d|3[01])T[0-2]\d:\d{2}:\d{2}.000Z)(?=\"))'```


In order to extract the texts data from all the texts we use a named group called ```created_at``` to capture all the datetime data.We use a <b>positive look behind</b> to get the text preceded by ```"created_at":```, followed by a <b>non-capturing group</b> that matches either 1 or 2 as the first digit followed by three digits, then a hyphen. The next non-capturing group captures two digit numbers such in the range 01-12 using the <b>alternation</b> to choose between 01-09 range and 10-12 range. This is done to match the 12 months of the year. After the next hyphen(literal match) we create another non-capturing group which captures two digits in the range 01-31. Note the use of character classes and alternations to implement this regex. This is followed by T (literal match) and a range of 0-2 implemented in a character class. This is follwowed by a simple regex pattern to capture the rest of the datetime format. Note that this regex is designed to capture any legitimate date. For this assessment we could have matched the literal ```2020``` after ```created_at:``` but this was not done to enable reusability of regexes.


 

In [31]:
def get_tweet_ids(tweet_file):
    """Function returns a list of ids containing all the tweet ids in the text

    Args:
        tweet_file (str, mandatory) : a text string from which ids need to be extracted.

    Returns:
        list : List of ids in a text"""
    
    # create and compile a regex
    id_pattern = r'(?P<id>(?<="id":\")\d+(?=\"))'
    regx = re.compile(id_pattern)
    
    # create a list of matched dictionaries
    id_list = [m.groupdict() for m in regx.finditer(tweet_file)]
    
    return id_list

# test
id_matches = get_tweet_ids(tweet_data_sample[0])
print(f"The first id of the text is : {id_matches[0]['id']}")

The first id of the text is : 1258367403953500164


In [32]:
def get_tweet_texts(tweet_file):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.

    Returns:
        list : List of texts in a tweet text"""
    
    # create and compile a regex
    text_pattern = r'(?P<text>(?<="text":\")[^"].*?(?=\"[,}][^\s\d]))'
    regx = re.compile(text_pattern)
    
    # create a list of matched dictionaries
    text_list = [m.groupdict() for m in regx.finditer(tweet_file)]
    
    return text_list

# test
text_matches = get_tweet_texts(tweet_data_sample[0])
print(f"The first text is : {text_matches[0]['text']}")

The first text is : @thehill Trump’s Ridiculous Behavior about Covid19 at Pandemic Briefings Battles https://t.co/4Gjy5tQ4XI https://t.co/8vcxU75Nz4


In [33]:
def get_tweet_timedates(tweet_file):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.

    Returns:
        list : List of texts in a tweet text"""
    
    # create and compile a regex
    created_at_pattern = r'(?P<created_at>(?<="created_at":\")(?:[12]\d{3}-(?:0[1-9]|1[0-2])-(?:0[1-9]|[12]\d|3[01])T[0-2]\d:\d{2}:\d{2}.000Z)(?=\"))'
    regx = re.compile(created_at_pattern)
    
    # create a list of matched dictionaries
    datetime_list = [m.groupdict() for m in regx.finditer(tweet_file)]
    
    return datetime_list

# test
datetime_matches = get_tweet_timedates(tweet_data_sample[0])
print(f"The first text is : {datetime_matches[0]['created_at']}")

The first text is : 2020-05-07T12:05:43.000Z


In all the above functions note the use of ```matchobject.groupdict()``` method which returns a dictionary of the named group as key and the match object as the value. The regexes were designed to take advantage of this method from the ```re``` package (hence the use of named dictionaries in the regexes). 

In the function below we use the three dictionaries we obtain from the above functions to recreate each tweet as a list of dictionaries. We make sure to check using Python's ```assert``` that each dictionary is of the same length to ensure we matching the appropriate id, created_at and text together. Incase the lengths are not matching an ```AssertionError``` is thrown with the file number that is causing the issue. 

In [34]:
def get_full_tweets(tweet_data):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.

    Returns:
        list : List of dictionary containing tweets"""
    
    # Initiate an empty list to capture all the tweets
    # file_num keeps track of file number for debugging purposes
    tweet_list = []
    file_num = 0
    
    # Iterate through each file
    for tweet_file in tweet_data:
        file_num += 1
        id_matches = get_tweet_ids(tweet_file)
        text_matches = get_tweet_texts(tweet_file)
        datetime_matches = get_tweet_timedates(tweet_file)
        
        # Make sure all lengths match
        # If they don't use the file number in the error for debugging
        assert len(id_matches) == len(text_matches) == len(datetime_matches), f"length not the same! check file index : {file_num+1}"
        
        # Append the other two dictionaries to one dictionary from above
        # We append text_matches and datetime_matches to id_matches dictionary
        for i in range(len(id_matches)):
            id_matches[i].update(text_matches[i])
            id_matches[i].update(datetime_matches[i])
        tweet_list.append(id_matches)
        
    return tweet_list

# Test
tweets_sample = get_full_tweets(tweet_data_sample)

In [35]:
tweets_sample[0]

[{'id': '1258367403953500164',
  'text': '@thehill Trump’s Ridiculous Behavior about Covid19 at Pandemic Briefings Battles https://t.co/4Gjy5tQ4XI https://t.co/8vcxU75Nz4',
  'created_at': '2020-05-07T12:05:43.000Z'},
 {'id': '1258367404029153280',
  'text': "Coronavirus : le plan pour la culture d'Emmanuel Macron https://t.co/0vUpUoXC1O",
  'created_at': '2020-05-07T12:05:43.000Z'},
 {'id': '1258367405157466112',
  'text': '‘Complacent’ UK draws global criticism for Covid-19 response https://t.co/Qmd2GSQPY5',
  'created_at': '2020-05-07T12:05:43.000Z'},
 {'id': '1258367406243745792',
  'text': 'Quatro hospitais particulares de Fortaleza estão com capacidade esgotada na urgência e emergência | Coronavírus - Últimas Notícias sobre o Coronavírus | O POVO Online https://t.co/y9w5t4Dbs9',
  'created_at': '2020-05-07T12:05:43.000Z'},
 {'id': '1258367406285762560',
  'text': "@Eglisecatho @Diocese72 +55 (22)981565421\\nDon't you have a BRAIN?\\nOnly (martial law)HOME PRISON, during 30 days, 

Below we implement a check to make sure we are on the right track. This simple function and assert statement are there to alert us if we extract fewer than <b>1172</b> tweets ( which are the total number tweets in the sample data). Python's ```eval``` function was used to find the total number of tweets. Although ```eval``` is unreliable most times when dealing with text data, it works for this purpose to just get the number of tweets in the sample. 

In [36]:
tweet_data_sam = []
for text in sample_data:
    data_pattern = r"\[\{.*\:.*\}\]\,"
    matches = re.search(data_pattern, text)
    _data = matches.group()[:-1]
    _data = eval(_data)
    tweet_data_sam.append(_data)

lens = []
for text in tweet_data_sam:
    lens.append(len(text))

print(f"There are {sum(lens)} tweets in the sample data")

There are 1172 tweets in the sample data


In [37]:
def sanity_check(list_of_dicts):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.

    Returns:
        list : List of texts in a tweet text"""    
    
    lengths = []
    i = 0
    for element in list_of_dicts:
        lengths.append(len(element))
    return sum(lengths)

assert sanity_check(tweets_sample) == 1172, "Total number of tweets not matching! \
                                            there should 1172 tweets! "

## 5. Create a date dictionary
In this next step we create a nested dictionary which has the unique dates in the dataset as the keys while in the dictionary, the ```id``` of the tweet is the key and ```text``` of the tweet is the value of the dictionary. In order to create this, the function  initialises an empty dictionary. We iterate through each list element looking for unique date matches which become the key of this new dictionary. 

<br>
<center><b><font size=5, color="blue"> Text Preprocessing </font></b></center>
<br>

Before we add the text to inner dictionary as the value, we need to preprocess it inorder to match the output of the sample file. Here is the list of preprocessing performed :

* Surrogate pairs
In order to refer the characters outside of the initial [BMP](https://en.wikipedia.org/wiki/Plane_(Unicode)#Basic_Multilingual_Plane), Universal Character Sets(UCS) use a pair containing a high surrogate (D800–DBFF) and a low surrogate(DC00–DFFF). Examples of these characters appearing in our text are \\uDDF2, \\uD83C, \\uDDEB etc. These are emoji encodings and these characters mislead the classifier and give us the wrong sample output. In order to avoid this, a regex is designed to capture these surrogate pairs. 


* ```r'\\*uD[8A-F][A-F0-9]{2}'```
This regex captures anything that starts with a ```\uD``` followed by either 8 or an alphabet between A and F then followed by two characters which could be between A-F and 0-9. This logic is implemented using character classes and note the use of ```{}``` to indicate exactly two characters.


* We then go on to replace quotation marks, apostrophes, less than, greater than with  XML's predefined entities (note that we don't replace ampersand because it is automatically being encoded while writing the file). We also replace the ```\\n``` with ```\n``` to match the sample output and enable formatting in our XML output


* Following this, encoding is done to allow surrogate pairs and the text is then classified. The text is now finally added to our dictionary.

In [38]:
def get_date_dict(tweets):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.

    Returns:
        list : List of texts in a tweet text"""
    
    classified_tweets = {}

    for list_elm in tweets:
        for tweet in list_elm:
            date_pattern = r'([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))'
            date = re.search(date_pattern, tweet['created_at']).group()

            if date not in classified_tweets:
                classified_tweets[date] = {}

            tweet_text = tweet['text']
            tweet_id = tweet['id']

            # Replace troublesome characters 
            tweet_text = re.sub(r'\\*uD[8A-F][A-F0-9]{2}', \
                              lambda x:eval('"' + x.group() + '"'), tweet_text)
            tweet_text = re.sub(r'\\"', "&quot;", tweet_text)
            tweet_text = re.sub(r"\'", "&apos;", tweet_text)
            tweet_text = re.sub(r"<", "&lt;", tweet_text)
            tweet_text = re.sub(r">", "&gt;", tweet_text)
            tweet_text = re.sub(r'\\n', '\n', tweet_text)

            # enable surrogate pass
            tweet_text = tweet_text.encode('utf-16', 'surrogatepass').decode('utf-16')

            # This functions language code and probability score in a tuple
            # Classify the text using the function
            language, prob = classify(tweet_text)

            if language == 'en':
                classified_tweets[date][tweet_id] = tweet_text

    return classified_tweets

english_tweets_sample = get_date_dict(tweets_sample)

In [39]:
english_tweets_sample

{'2020-05-07': {'1258367403953500164': '@thehill Trump’s Ridiculous Behavior about Covid19 at Pandemic Briefings Battles https://t.co/4Gjy5tQ4XI https://t.co/8vcxU75Nz4',
  '1258367405157466112': '‘Complacent’ UK draws global criticism for Covid-19 response https://t.co/Qmd2GSQPY5',
  '1258367406285762560': '@Eglisecatho @Diocese72 +55 (22)981565421\nDon&apos;t you have a BRAIN?\nOnly (martial law)HOME PRISON, during 30 days, of all(100%) CITIZENS will stop CORONA VIRUS.\nNo more deaths.\nThe economy will be normal in 30 days too.\n CURE See Dra Eunara Negri discovered the CURE of CORONAVIRUS.\nhttps://t.co/2uLsT6Eehn',
  '1258367407606960129': 'UK contact-tracing app could fall foul of privacy law, government told\n\nhttps://t.co/W85NfMKadm',
  '1258367407653040133': 'Do you want to be up to date with #coronavirus worldwide? It is a good idea to attach this 👇 page to your sources. https://t.co/1mh5p6IQFz',
  '1258367407770472449': '2 Utah County businesses told staff to ignore COVID-1

In [40]:
lens = []
for key in english_tweets_sample:
      lens.append(len(english_tweets_sample[key]))
assert sum(lens) == 697, f"Something is wrong! You should have 697 total tweets,\
                                                                        not {lens}."

## 6. Create the XML file
The function below writes an XML file to the current directory depending on the kind of data supplied to it. If the data given is ```sample```, the output is XML of sample data whereas if the ```full``` data is given, the output is XML of full data. 

In [41]:
# creating xml file
def create_xml(classified_tweets, data_type='sample'):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.
        data_type (str, optional) : a tweet string from which text need to be extracted
    Returns:
        list : List of texts in a tweet text"""

    accepted_dataypes = ['sample', 'full']
    assert data_type in accepted_dataypes, "Enter a valid data type!"

    if data_type == 'sample':
        file_name =  './sample_result.xml'
    else:
        file_name = './30914345.xml'

    with open(file_name, 'w', encoding="UTF-8") as f:
        f.write(f"""<?xml version="1.0" encoding="UTF-8"?>\n""")
        f.write("<data>\n")
        for tweet_date, tweets in classified_tweets.items():
            f.write(f"""<tweets date="{tweet_date}">\n""")
            for tweet in tweets.items():
                f.write(f"""<tweet id="{tweet[0]}">{tweet[1]}</tweet>\n""")
            f.write(f"""</tweets>\n""")
        f.write("</data>\n")

create_xml(english_tweets_sample)

## 7. Unified function to extract english tweets
Here we create a unified function that can extract all the english tweets from a directory containing text files of tweets. The input to the function is the ```file_path``` to the directory and the ```data_type``` we want to process. Note that ```sample``` is a default argument for the function, ie, if we give the path of full files and do not mention the ```data_type``` to be ```full``` the output XML is created using sample data. This is because running full data takes a lot of compute and we want to avoid accidental runs of this function. <b> So unless the keyword argument data_type is equal to "full", the full data is not processed. </b>

In [42]:

def get_english_tweets(file_path, data_type='sample'):
    """Function returns a list of texts containing all the tweet texts in the text

    Args:
        tweet_file (str, mandatory) : a tweet string from which text need to be extracted.

    Returns:
        list : List of texts in a tweet text"""
    
    list_of_texts = read_files(file_path)
    tweet_data = grab_data_tag(list_of_texts)
    tweets = get_full_tweets(tweet_data)
    classified_tweets = get_date_dict(tweets)
    create_xml(classified_tweets, data_type)
    
    if data_type == 'sample':
        message = 'Check your current directory for the output XML named sample_result.xml'
    else:
        message = 'Check your current directory for the output XML named 30914345.xml'
    return message

## Process the full data 
We now run the unified function ```get_english_tweets``` on the full data and this returns an XML file in the current directory. 

In [43]:
# Run the unified function on full data
file_path_full = './30914345'
get_english_tweets(file_path_full, data_type='full')

'Check your current directory for the output XML named 30914345.xml'

## References
( Referencing style : [APA 6th Referencing Style Guide](https://aut.ac.nz.libguides.com/APA6th/websites ) )

* ```os``` library methods. os documentation : os.path, os.listdir. Retrieved August 22, 2020, from
https://docs.python.org/3/library/os.html


* ```re``` library methods. re documentation : re.sub, re.match, re.findall. Retrieved August 25, 2020, from
https://docs.python.org/3/library/re.html


* ```langid``` library function. langid documentation : langid.classify. Retrieved August 27, 2020, from
https://pypi.org/project/langid/1.1.5/


* Jan Goyvaerts (2013-2020), How to build regular expressions. Retrieved September 7, 2020, from
https://www.regular-expressions.info/


* XML's predefined entities (17 July 2020). Wikipedia, the free encyclopedia. Retrieved September 8, 2020
https://en.wikipedia.org/wiki/List_of_XML_and_HTML_character_entity_references#Predefined_entities_in_XML


* Surrogates and Universal Character Sets (13 June 2020). Wikipedia, the free encyclopedia. Retrieved September 9, 2020, from https://en.wikipedia.org/wiki/Universal_Character_Set_characters#Surrogates
